# Ollama Function Calling
- By Marcelo Rovai @17Oct25

## Importing Libraries

- **haversine**: A function from the haversine library that calculates the distance between two geographic points using the Haversine formula.

In [7]:
import time
from haversine import haversine
from ollama import chat

## Defining Input and Model

On a python string, it is possible get the country from command-line arguments

In this notebook, we should define the country name as a variable

In [57]:
country = "Colombia"

In [58]:
MODEL = 'llama3.2:3B'   # The name of the model to be used
mylat = -33.33          # Latitude of Santiago de Chile
mylon = -70.51          # Longitude of Santiago de Chile

## Defining a Python Function That Acts as a Tool

In [59]:
def calc_distance(lat, lon, city):
    """Compute distance and print a descriptive message."""
    distance = haversine((mylat, mylon), (lat, lon), unit="km")
    msg = f"\nSantiago de Chile is about {int(round(distance, -1)):,} kilometers away from {city}."
    return {"city": city, "distance_km": int(round(distance, -1)), "message": msg}

## Declaring the tool descriptor (schema)

In [60]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "calc_distance",
            "description": "Calculates the distance from Santiago, Chile to a given city's coordinates.",
            "parameters": {
                "type": "object",
                "properties": {
                    "lat": {"type": "number", "description": "Latitude of the city"},
                    "lon": {"type": "number", "description": "Longitude of the city"},
                    "city": {"type": "string", "description": "Name of the city"}
                },
                "required": ["lat", "lon", "city"]
            }
        }
    }
]

## Asking the Model to Use the Tool

In [61]:
def ask_and_measure(country):
    """Let Ollama find the capital and call the local Python function."""
    start = time.perf_counter()

    response = chat(
        model=MODEL,
        messages=[{
            "role": "user",
            "content": f"Find the decimal latitude and longitude of the capital of {country},"
                       " then use the calc_distance tool to determine how far it is from Santiago de Chile."
        }],
        tools=tools
    )
    return response

In [62]:
response = ask_and_measure(country)
response

ChatResponse(model='llama3.2:3B', created_at='2025-10-17T18:27:49.842521004Z', done=True, done_reason='stop', total_duration=9196115725, load_duration=207209859, prompt_eval_count=223, prompt_eval_duration=2668307535, eval_count=32, eval_duration=6319895710, message=Message(role='assistant', content='', thinking=None, images=None, tool_name=None, tool_calls=[ToolCall(function=Function(name='calc_distance', arguments={'city': 'Bogota', 'lat': 4.6583, 'lon': -74.0961}))]))

In [63]:
resp = response.message.tool_calls
resp[0]

ToolCall(function=Function(name='calc_distance', arguments={'city': 'Bogota', 'lat': 4.6583, 'lon': -74.0961}))

In [69]:
for call in response.message.tool_calls:
    raw_args = call["function"]["arguments"]
raw_args

{'city': 'Bogota', 'lat': 4.6583, 'lon': -74.0961}

In [70]:
city = raw_args['city'] 
lat = float(raw_args['lat'])
lon = float(raw_args['lon'])
city, lat, lon

('Bogota', 4.6583, -74.0961)

## Calculating the Distance

In [71]:
distance = haversine((mylat, mylon), (lat, lon), unit='km')
distance

4240.961800075008

In [72]:
print(f"Santiago de Chile is about {int(round(distance, -1)):,} kilometers away from {city}.")

Santiago de Chile is about 4,240 kilometers away from Bogota.


## Creating a function

In [73]:
def calc_dist(country, model=MODEL):
    
    start_time = time.perf_counter()  # Start timing

    # Ask Ollama for structured data
    """Let Ollama find the capital and call the local Python function."""
    start = time.perf_counter()

    response = chat(
        model=MODEL,
        messages=[{
            "role": "user",
            "content": f"Find the decimal latitude and longitude of the capital of {country},"
                       " then use the calc_distance tool to determine how far it is from Santiago de Chile."
        }],
        tools=tools
    )

    for call in response.message.tool_calls:
        raw_args = call["function"]["arguments"]
    city = raw_args['city'] 
    lat = float(raw_args['lat'])
    lon = float(raw_args['lon'])
    
    distance = haversine((mylat, mylon), (lat, lon), unit='km')
    
    end_time = time.perf_counter()  # End timing
    elapsed_time = end_time - start_time  # Calculate elapsed time
    
    print(f"\nSantiago de Chile is about {int(round(distance, -1)):,} kilometers away from {city}.") 
    print(f"[INFO] ==> {MODEL}: {elapsed_time:.1f} seconds")


In [74]:
calc_dist('france', model=MODEL)


Santiago de Chile is about 11,630 kilometers away from Paris.
[INFO] ==> llama3.2:3B: 9.7 seconds


In [75]:
calc_dist('colombia')


Santiago de Chile is about 4,240 kilometers away from Bogotá.
[INFO] ==> llama3.2:3B: 10.4 seconds


In [76]:
calc_dist('united states')


Santiago de Chile is about 8,290 kilometers away from United States.
[INFO] ==> llama3.2:3B: 7.3 seconds


### Using Llama3.2:1B

In [77]:
MODEL = 'llama3.2:1B'
calc_dist('france')
calc_dist('colombia')
calc_dist('united states')


Santiago de Chile is about 10,990 kilometers away from Paris.
[INFO] ==> llama3.2:1B: 29.9 seconds

Santiago de Chile is about 4,260 kilometers away from Bogotá.
[INFO] ==> llama3.2:1B: 8.4 seconds

Santiago de Chile is about 8,160 kilometers away from Washington.
[INFO] ==> llama3.2:1B: 6.3 seconds


### Using gemma3n:e2b

In [78]:
MODEL = 'gemma3n:e2b'
calc_dist('france')
calc_dist('colombia')
calc_dist('united states')

ResponseError: registry.ollama.ai/library/gemma3n:e2b does not support tools (status code: 400)

### Using gemma3:4b

In [79]:
MODEL = 'gemma3:4b'
calc_dist('france')
calc_dist('colombia')
calc_dist('united states')

ResponseError: registry.ollama.ai/library/gemma3:4b does not support tools (status code: 400)